In [2]:
import pandas as pd
import nltk
import numpy as np
from functools import reduce
import math


In [3]:
NOME_DO_ARQUIVO = 'estadao_noticias_eleicao.csv'
indice_invertido = {}
dados = pd.read_csv(NOME_DO_ARQUIVO)
dados = dados.replace(np.nan, '', regex=True)
dados["TF"] = None

In [36]:
def geraIndice():
    for j in range(dados.titulo.size):
        i = j + 1
        texto = dados.titulo[i] + " " + dados.conteudo[i] + " " + dados.subTitulo[i]
        palavras_divididas = nltk.word_tokenize(texto)
        dados.TF[i] = {}
        for palavra in palavras_divididas:
            palavra = palavra.lower()
            if palavra not in indice_invertido:
                indice_invertido[palavra] = set([dados.idNoticia[i]])
            else:
                indice_invertido[palavra].add(dados.idNoticia[i])
            if palavra not in dados.TF[i]:
                dados.TF[i][palavra] = 0
            dados.TF[i][palavra] += 1

In [5]:
def modeloBinario(termos):
    return list(reduce((lambda x,y: indice_invertido[x] & indice_invertido[y]), termos))

In [6]:
def getDocumentsWithTerms(terms):
    return list(reduce((lambda x,y: indice_invertido[x].union(indice_invertido[y])), terms))

In [7]:
def generateQueryTF(query):
    tf = {}
    for term in query:
        if term not in tf:
            tf[term] = 0
        tf[term] += 1
    return tf

In [15]:
def modeloTF(termos):
    documentos = modeloBinario(termos)
    documentsTF = []
    for idDoc in documentos:
        documentWeight = 0
        for termo in termos:
            docTF = dados.TF[idDoc][termo] if (termo in dados.TF[idDoc]) else 0
            qTF = termos.count(termo)
            documentWeight += docTF * qTF
            
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

In [21]:
def modeloIDF(termos):
    documentos = modeloBinario(termos)
    documentsTF = []
    for idDoc in documentos:
        documentWeight = 0
        for termo in termos:
            docTF = dados.TF[idDoc][termo] if (termo in dados.TF[idDoc]) else 0
            qTF = termos.count(termo)
            idf = math.log(dados.titulo.size/len(indice_invertido[termo]))
            documentWeight += docTF * qTF * idf
            
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

In [ ]:
geraIndice()
termos = ["segundo", "turno"]
consultaTF = modeloTF(termos)[:5]
consultaIDF = modeloIDF(termos)[:5]

[2744, 7, 2112, 7672, 2388]
[2744, 2112, 7672, 1235, 2388]

print(consultaTF)
print(consultaIDF)

/opt/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
dados.TF[0]["dilma"]

9